## NCAA March Madness: Sabermetric Spin Second Edition

## Read Me

This is the second version of my NCAA March Madness: Sabermetric Spin series for the March Machine Learning Mania predictions tournament. Special credits go to the following notebook: https://www.kaggle.com/theoviel/ncaa-starter-the-simpler-the-better. You can find the previous version linked at https://www.kaggle.com/code/toshimelonhead/ncaa-march-madness-sabermetric-spin.

I completely rewrote my original version to adapt for the new parts of the 2023 challenge (new loss function, combining men's and women's scores). Also included this year is Elo Ratings, Pythagorean scores, and better visualizations. I figured this should be public because many of these ideas are in other notebooks, but not combined in one spot. Here's to hoping we produce a better result than last season!

## Loading Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import optuna
import re
import sklearn

from optuna.visualization import plot_slice
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import *
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.tree import *
from sklearn.svm import SVC, LinearSVC


import xgboost as xgb
import lightgbm as lgb

## Loading Dataset

In [ ]:
DATA_PATH = '../input/march-machine-learning-mania-2023/'

data = dict()
for filename in os.listdir(DATA_PATH):
    print(filename)
    try:
        data[f'{filename[:-4]}'] = pd.read_csv(DATA_PATH + filename, engine='python')
    except UnicodeDecodeError:
        print(filename, 'not loaded')
        continue
    

## Exploratory Data Analysis: First Glance

### Seeds

In [ ]:
df_seeds = pd.concat([data['MNCAATourneySeeds'], data['WNCAATourneySeeds']], ignore_index=True)

def treat_seed(seed):
    return int(re.sub("[^0-9]", "", seed))

df_seeds['Seed'] = df_seeds['Seed'].apply(treat_seed)
df_seeds.info()

In [ ]:
data['MTeams'].to_csv('MTeams.csv')

In [ ]:
df_seeds.tail() # We have 2023 seeds now, huzzah!

### Season Results

In [ ]:
men_season = data['MRegularSeasonDetailedResults']
women_season = data['WRegularSeasonDetailedResults']
men_season['Gender'] = 1
women_season['Gender'] = 0 # It's boolean, not bias
df_season_results = pd.concat([men_season, women_season])
df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

df_season_results.head()
df_season_results[['Season', 'WTeamID', 'LTeamID']] = df_season_results[['Season', 'WTeamID', 'LTeamID']].astype(str)

In [ ]:
df_teams = pd.concat([data['MTeams'][['TeamID', 'TeamName']], data['WTeams']], axis=0)

In [ ]:
df_teams.head()

### Tourney Results

In [ ]:
men_tourney = data['MNCAATourneyDetailedResults']
women_tourney = data['WNCAATourneyDetailedResults']
men_tourney['Gender'] = 1
women_tourney['Gender'] = 0 
df_tourney_results = pd.concat([men_tourney, women_tourney])
df_tourney_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)
df_tourney_results[['Season', 'WTeamID', 'LTeamID']] = df_tourney_results[['Season', 'WTeamID', 'LTeamID']].astype(str)
df_tourney_results.head()

## Compute Wins, Losses, Scoring Margin, Pythagorean W/L

### Wins and Losses

In [ ]:
num_win = df_season_results.groupby(['Season', 'WTeamID']).count()
num_win = num_win.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"}).fillna(0)

num_loss = df_season_results.groupby(['Season', 'LTeamID']).count()
num_loss = num_loss.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"}).fillna(0)


In [ ]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

df_features_season = pd.concat([df_features_season_w, df_features_season_l], axis=0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')

df_features_season['NumWins'] = df_features_season['NumWins'].fillna(0)
df_features_season['NumLosses'] = df_features_season['NumLosses'].fillna(0)


df_features_season['WinPct'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])

In [ ]:
df_features_season.info()

### Scoring Margin

In [ ]:
df_season_results['ScoreMargin'] = df_season_results['WScore'] - df_season_results['LScore']

win_score_margin = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()
win_score_margin = win_score_margin[['Season', 'WTeamID', 'ScoreMargin']].rename(columns={"ScoreMargin": "AvgWinningScoreMargin", "WTeamID": "TeamID"}).fillna(0)

lose_score_margin = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()
lose_score_margin = lose_score_margin[['Season', 'LTeamID', 'ScoreMargin']].rename(columns={"ScoreMargin": "AvgLosingScoreMargin", "LTeamID": "TeamID"}).fillna(0)

df_features_season = df_features_season.merge(win_score_margin, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(lose_score_margin, on=['Season', 'TeamID'], how='left')
df_features_season.fillna(0, inplace=True) # Takes care of undefeated teams


df_features_season['AvgScoringMargin'] = (
    (df_features_season['NumWins'] * df_features_season['AvgWinningScoreMargin'] - 
    df_features_season['NumLosses'] * df_features_season['AvgLosingScoreMargin'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)

### Points Scored and Allowed

In [ ]:
win_points_scored = df_season_results.groupby(['Season', 'WTeamID'])['WScore'].sum().reset_index().rename(columns={"WScore": "WPointsScored", "WTeamID": "TeamID"})
lose_points_scored = df_season_results.groupby(['Season', 'LTeamID'])['LScore'].sum().reset_index().rename(columns={"LScore": "LPointsScored", "LTeamID": "TeamID"})

win_points_allowed = df_season_results.groupby(['Season', 'WTeamID'])['LScore'].sum().reset_index().rename(columns={"LScore": "WPointsAllowed", "WTeamID": "TeamID"})
lose_points_allowed = df_season_results.groupby(['Season', 'LTeamID'])['WScore'].sum().reset_index().rename(columns={"WScore": "LPointsAllowed", "LTeamID": "TeamID"})

In [ ]:
total_points_scored = win_points_scored.merge(lose_points_scored, how='outer').fillna(0)
total_points_scored['PointsScored'] = total_points_scored['WPointsScored'] + total_points_scored['LPointsScored']

total_points_allowed = win_points_allowed.merge(lose_points_allowed, how='outer').fillna(0)
total_points_allowed['PointsAllowed'] = total_points_allowed['WPointsAllowed'] + total_points_allowed['LPointsAllowed']

In [ ]:
total_points_allowed.head()

### Pythagorean W/L

In [ ]:
total_points = total_points_scored.merge(total_points_allowed, how='outer')
total_points['Pythagorean'] = 1 / (1 + (total_points['PointsAllowed'] / total_points['PointsScored']) ** 8)

# The 8 is arbitrary (KenPom I think uses 11.5). I wanted something that would create a nice bell shaped curve like below.
sns.histplot(total_points['Pythagorean'])
plt.show()

In [ ]:
df_season_condensed = df_season_results[['Season', 'WTeamID', 'LTeamID']].copy()

df_season_condensed_pyth = df_season_condensed.merge(total_points[['Season', 'TeamID', 'Pythagorean']], left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left', suffixes=[None, "W"])
df_season_condensed_pyth = df_season_condensed_pyth.merge(total_points[['Season', 'TeamID', 'Pythagorean']], left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left', suffixes=[None, 'L'])
df_season_condensed_pyth['PythagoreanDiff'] = df_season_condensed_pyth['Pythagorean'] - df_season_condensed_pyth['PythagoreanL']

sns.histplot(df_season_condensed_pyth['PythagoreanDiff'])
plt.axvline(0, color='red')
plt.show()

### Team Trend Over Time: The Tennessee Correction

This corrects for team changes over time such as injuries late in the season. It also rewards teams that are playing well compared to their record heading into the tournament.

In [ ]:
# Find where to split the number of games into 75-25 split.
split = np.percentile(df_season_results['DayNum'], 75)
first_three_quarters = df_season_results[df_season_results['DayNum'] < split].copy()
fourth_quarter = df_season_results[df_season_results['DayNum'] >= split].copy()

In [ ]:
def pythagorean(df, exponent=8):
    """Computes the pythagorean percentage given a number of games played in the season."""
    
    win_points_scored = df.groupby(['Season', 'WTeamID'])['WScore'].sum().reset_index().rename(columns={"WScore": "WPointsScored", "WTeamID": "TeamID"})
    lose_points_scored = df.groupby(['Season', 'LTeamID'])['LScore'].sum().reset_index().rename(columns={"LScore": "LPointsScored", "LTeamID": "TeamID"})

    win_points_allowed = df.groupby(['Season', 'WTeamID'])['LScore'].sum().reset_index().rename(columns={"LScore": "WPointsAllowed", "WTeamID": "TeamID"})
    lose_points_allowed = df.groupby(['Season', 'LTeamID'])['WScore'].sum().reset_index().rename(columns={"WScore": "LPointsAllowed", "LTeamID": "TeamID"})

    total_points_scored = win_points_scored.merge(lose_points_scored, how='outer').fillna(0)
    total_points_scored['PointsScored'] = total_points_scored['WPointsScored'] + total_points_scored['LPointsScored']

    total_points_allowed = win_points_allowed.merge(lose_points_allowed, how='outer').fillna(0)
    total_points_allowed['PointsAllowed'] = total_points_allowed['WPointsAllowed'] + total_points_allowed['LPointsAllowed']

    total_points = total_points_scored.merge(total_points_allowed, how='outer')
    total_points['Pythagorean'] = 1 / (1 + (total_points['PointsAllowed'] / total_points['PointsScored']) ** 8)

    # The 8 is arbitrary (KenPom I think uses 11.5). I wanted something that would create a nice bell shaped curve like below.
    return total_points

In [ ]:
pyth_first_three_quarters = pythagorean(first_three_quarters)
pyth_fourth_quarter = pythagorean(fourth_quarter)
pyth_by_period = pyth_first_three_quarters.merge(pyth_fourth_quarter,
                                    on=['Season', 'TeamID'], 
                                    suffixes=['_A', '_B'])\
                                    .set_index(['Season', 'TeamID'])[['Pythagorean_A', 'Pythagorean_B']]\
                                    .sort_values(['Season', 'TeamID'])

pyth_by_period.columns = ['Early', 'Late']
pyth_by_period['Trend'] = pyth_by_period['Late'] - pyth_by_period['Early']
pyth_by_period['Trend'].fillna(0)

## Sabermetrics

In [ ]:
sabermetrics = pd.DataFrame()
sabermetrics_differences = pd.DataFrame()

sabermetrics['Season'] = df_season_results['Season']
sabermetrics['WTeamID'] = df_season_results['WTeamID']
sabermetrics['LTeamID'] = df_season_results['LTeamID']

sabermetrics_differences['Season'] = df_season_results['Season']
sabermetrics_differences['WTeamID'] = df_season_results['WTeamID']
sabermetrics_differences['LTeamID'] = df_season_results['LTeamID']

# Number of Possessions
sabermetrics['WPossessions'] = (df_season_results['WFGA'] - df_season_results['WOR']) + df_season_results['WTO'] + .44 * df_season_results['WFTA']
sabermetrics['LPossessions'] = (df_season_results['LFGA'] - df_season_results['LOR']) + df_season_results['LTO'] + .44 * df_season_results['LFTA']

df_season_results['WPossessions'] = sabermetrics['WPossessions']
df_season_results['LPossessions'] = sabermetrics['LPossessions']

# Points Per Possession
sabermetrics['WPtsPerPoss'] = df_season_results['WScore'] / df_season_results['WPossessions']
sabermetrics['LPtsPerPoss'] = df_season_results['LScore'] / df_season_results['LPossessions']

# Opponent Points Per Possession:
sabermetrics['WOPtsPerPoss'] = df_season_results['LScore'] / df_season_results['LPossessions'] # O for 'opponent'
sabermetrics['LOPtsPerPoss'] = df_season_results['WScore'] / df_season_results['WPossessions']

# Effective Field Goal Percentage
sabermetrics['WEffectiveFGPct'] = ((df_season_results['WScore'] - df_season_results['WFTM']) / 2) / df_season_results['WFGA']
sabermetrics['LEffectiveFGPct'] = ((df_season_results['LScore'] - df_season_results['LFTM']) / 2) / df_season_results['LFGA']

# Percentage of Field Goals Assisted
sabermetrics['WAssistRate'] = df_season_results['WAst'] / df_season_results['WFGM']
sabermetrics['LAssistRate'] = df_season_results['LAst'] / df_season_results['LFGM']

# Rebound Percentage
sabermetrics['WReboundPct'] = (df_season_results['WDR'] + df_season_results['WOR']) / (df_season_results['LFGA'] - df_season_results['LFGM'])
sabermetrics['LReboundPct'] = (df_season_results['LDR'] + df_season_results['LOR']) / (df_season_results['WFGA'] - df_season_results['WFGM'])

# Steal Block Foul Ratio (the plus 10's are for getting rid of infinity and 0 bounds)
sabermetrics['WStealBlockFoul'] = (df_season_results['WStl'] + df_season_results['WBlk'] + 10) / (df_season_results['WPF'] + 10)
sabermetrics['LStealBlockFoul'] = (df_season_results['LStl'] + df_season_results['LBlk'] + 10) / (df_season_results['LPF'] + 10)

# Turnover Rate
sabermetrics['WTORate'] = df_season_results['WTO'] / df_season_results['WPossessions']
sabermetrics['LTORate'] = df_season_results['LTO'] /  df_season_results['LPossessions']

# Percentage of Shots Beyond the Arc
sabermetrics['WBArcPct'] = df_season_results['WFGA3'] / df_season_results['WFGA']
sabermetrics['LBArcPct'] = df_season_results['LFGA3'] /  df_season_results['LFGA']

# Free Throw Rate
sabermetrics['WFTRate'] = df_season_results['WFTA'] / df_season_results['WFGA']
sabermetrics['LFTRate'] = df_season_results['LFTA'] /  df_season_results['LFGA']

# Free Throw Percentage
sabermetrics['WFTPct'] = df_season_results['WFTM'] / df_season_results['WFTA']
sabermetrics['LFTPct'] = df_season_results['LFTM'] / df_season_results['LFTA']

In [ ]:
# Check for missing values
sabermetrics.info()

In [ ]:
# Fill in missing values
sabermetrics['WStealBlockFoul'].fillna(df_season_results['WStl'] + df_season_results['WBlk'], inplace=True)
sabermetrics.fillna(0, inplace=True)

# Differences (used for plotting below)
sabermetrics['PtsPerPossDiff'] = sabermetrics['WPtsPerPoss'] - sabermetrics['LPtsPerPoss']
sabermetrics['OPtsPerPossDiff'] = sabermetrics['WOPtsPerPoss'] - sabermetrics['LOPtsPerPoss'] # O for opponent
sabermetrics['EffectiveFGPctDiff'] = sabermetrics['WEffectiveFGPct'] - sabermetrics['LEffectiveFGPct']
sabermetrics['AssistRateDiff'] = sabermetrics['WAssistRate'] - sabermetrics['LAssistRate']
sabermetrics['ReboundPctDiff'] = sabermetrics['WReboundPct'] - sabermetrics['LReboundPct']
sabermetrics['StealBlockFoulDiff'] = sabermetrics['WStealBlockFoul'] - sabermetrics['LStealBlockFoul']
sabermetrics['TORateDiff'] = sabermetrics['WTORate'] - sabermetrics['LTORate']
sabermetrics['BArcPctDiff'] = sabermetrics['WBArcPct'] - sabermetrics['LBArcPct']
sabermetrics['FTRateDiff'] = sabermetrics['WFTRate'] - sabermetrics['LFTRate']
sabermetrics['FTPctDiff'] = sabermetrics['WFTPct'] - sabermetrics['LFTPct']

In [ ]:
figs, axs = plt.subplots(nrows=5, ncols=2, squeeze=False, figsize=(12, 12))

sns.histplot(sabermetrics['PtsPerPossDiff'], ax=axs[0, 0])
axs[0, 0].set_xlabel('Points Per Possession Difference')
axs[0, 0].axvline(0, color='red')

sns.histplot(sabermetrics['OPtsPerPossDiff'], ax=axs[1, 0])
axs[1, 0].set_xlabel('Opponent Points Per Possession Difference')
axs[1, 0].axvline(0, color='red')

sns.histplot(sabermetrics['EffectiveFGPctDiff'], ax=axs[2, 0])
axs[2, 0].set_xlabel('Effective FG Pct Difference')
axs[2, 0].axvline(0, color='red')

sns.histplot(sabermetrics['AssistRateDiff'], ax=axs[3, 0])
axs[3, 0].set_xlabel('Assist Rate Difference')
axs[3, 0].axvline(0, color='red')

sns.histplot(sabermetrics['ReboundPctDiff'], ax=axs[4, 0])
axs[4, 0].set_xlabel('Rebound Pct Difference')
axs[4, 0].axvline(0, color='red')

sns.histplot(sabermetrics['TORateDiff'], ax=axs[0, 1])
axs[0, 1].set_xlabel('TO Rate Difference')
axs[0, 1].axvline(0, color='red')

sns.histplot(sabermetrics['BArcPctDiff'], ax=axs[1, 1])
axs[1, 1].set_xlabel('Three Point Attempt Rate Difference')
axs[1, 1].axvline(0, color='red')

sns.histplot(sabermetrics['FTRateDiff'], ax=axs[2, 1])
axs[2, 1].set_xlabel('Free Throw Attempt Rate Difference')
axs[2, 1].axvline(0, color='red')

sns.histplot(sabermetrics['FTPctDiff'], ax=axs[3, 1])
axs[3, 1].set_xlabel('Free Throw Percentage Difference')
axs[3, 1].axvline(0, color='red')

sns.histplot(sabermetrics['StealBlockFoulDiff'], ax=axs[4, 1])
axs[4, 1].set_xlabel('Steal Plus Block to Foul Ratio Difference')
axs[4, 1].axvline(0, color='red')
plt.tight_layout()
plt.show()

In [ ]:
winning_columns = sabermetrics[[col for col in sabermetrics.columns if col[0] == 'W']]
losing_columns = sabermetrics[[col for col in sabermetrics.columns if col[0] == 'L']]

winning_columns.loc[:, 'Season'] = sabermetrics['Season']
losing_columns.loc[:, 'Season'] = sabermetrics['Season']

winning_sabermetrics = winning_columns.groupby(['Season', 'WTeamID']).mean()
losing_sabermetrics = losing_columns.groupby(['Season', 'LTeamID']).mean()

winning_sabermetrics = winning_sabermetrics \
                        .reset_index() \
                        .merge(df_features_season[['Season', 'TeamID', 'NumWins']], left_on=['Season', 'WTeamID'], right_on=['Season', 'TeamID'], how='left') \
                        .set_index(['Season', 'WTeamID']) \
                        .drop(['TeamID'], axis=1)

losing_sabermetrics = losing_sabermetrics \
                        .reset_index() \
                        .merge(df_features_season[['Season', 'TeamID', 'NumLosses']], left_on=['Season', 'LTeamID'], right_on=['Season', 'TeamID'], how='left') \
                        .set_index(['Season', 'LTeamID']) \
                        .drop(['TeamID'], axis=1)

weighted_sabermetrics_wins = winning_sabermetrics[[col for col in winning_sabermetrics.columns if col[0] == 'W']].multiply(winning_sabermetrics['NumWins'], axis=0)
weighted_sabermetrics_losses = losing_sabermetrics[[col for col in losing_sabermetrics.columns if col[0] == 'L']].multiply(losing_sabermetrics['NumLosses'], axis=0)

weighted_sabermetrics = pd.DataFrame()
weighted_sabermetrics['Possessions'] = (weighted_sabermetrics_wins['WPossessions'] + weighted_sabermetrics_losses['LPossessions']) /  \
                                       (winning_sabermetrics['NumWins'] + losing_sabermetrics['NumLosses'])

combined_df = winning_sabermetrics.reset_index().merge(losing_sabermetrics.reset_index(), left_on=['WTeamID', 'Season'], right_on=['LTeamID', 'Season'], how='outer')

def weighted_metric(metric, df=combined_df.set_index(['Season', 'WTeamID'], inplace=True)):
    """Computes the weighted stat from winning and losing metric"""
        
    weighted_df = ((combined_df[f'W{metric}'].mul(combined_df['NumWins']) + combined_df[f'L{metric}'].mul(combined_df['NumLosses'])) \
    / (combined_df['NumWins'] + combined_df['NumLosses']))
    return weighted_df


combined_df.reset_index(inplace=True)
combined_df['WTeamID'].fillna(combined_df['LTeamID'], inplace=True)
combined_df['LTeamID'].fillna(combined_df['WTeamID'], inplace=True)
combined_df.set_index(['Season', 'WTeamID'], inplace=True)
combined_df.fillna(0, inplace=True)

metrics_list = ['PtsPerPoss', 'OPtsPerPoss', 'EffectiveFGPct', 'AssistRate', 'ReboundPct', 'TORate', 'BArcPct', 'FTRate', 'FTPct', 'StealBlockFoul']
season_sabermetrics = pd.concat([weighted_metric(metric) for metric in metrics_list], axis=1)
season_sabermetrics.columns=metrics_list

season_sabermetrics.index.columns = ['Season', 'TeamID']


## Elo Ratings

In [ ]:
def update_elo_scores(df, scores_dict, k=40):
    for index, row in df.iterrows():
        player1, player2 = str(row['WTeamID']), str(row['LTeamID'])
        if player1 not in scores_dict:
            scores_dict[player1] = 1000
        if player2 not in scores_dict:
            scores_dict[player2] = 1000
        score1, score2 = scores_dict[player1], scores_dict[player2]
        expected_score1 = 1 / (1 + 10 ** ((score2 - score1) / 400))
        expected_score2 = 1 / (1 + 10 ** ((score1 - score2) / 400))
        scores_dict[player1] = score1 + k * (1 - expected_score1)
        scores_dict[player2] = score2 + k * (0 - expected_score2)
    return scores_dict

all_seasons = dict()
scores_dict = dict()

for season in range(2003, 2024):
    df_season_condensed_year = df_season_condensed[df_season_condensed['Season'] == str(season)]
    scores_dict = update_elo_scores(df_season_condensed_year, scores_dict)
    all_seasons[str(season)] = scores_dict
    scores_dict = dict() # Reset dict for next season
    
elo_scores = pd.DataFrame(all_seasons)
elo_scores_unstacked = elo_scores.unstack().reset_index()
elo_scores_unstacked.columns = ['Season', 'TeamID', 'Elo']

sns.histplot(elo_scores_unstacked['Elo'])
plt.axvline(1000, color='red')
plt.show()

In [ ]:
elo_scores.head()

## Opponent Elo Ratings

In [ ]:
df_teams.head()

In [ ]:
all_opponent_elos = dict() # The dataframe to hold all Elo scores
all_opponent_elos_df = pd.DataFrame() # The final dataframe

for season in range(2003, 2024):
    df_season_condensed_year = df_season_condensed[df_season_condensed['Season'] == str(season)].reset_index(drop=True)
    df_season_condensed_year['WElo'] = df_season_condensed_year.merge(elo_scores[str(season)], left_on=['WTeamID'], right_index=True, how='inner')[str(season)]
    df_season_condensed_year['LElo'] = df_season_condensed_year.merge(elo_scores[str(season)], left_on=['LTeamID'], right_index=True, how='inner')[str(season)]
    opponent_elo_dicts = dict()
    for game in df_season_condensed_year.iterrows():
        # Create dictionary for each team to hold opponent Elo ratings
        
        if game[1]['WTeamID'] not in opponent_elo_dicts.keys():
            opponent_elo_dicts[str(game[1]['WTeamID'])] = []
        if game[1]['LTeamID'] not in opponent_elo_dicts.keys():
            opponent_elo_dicts[str(game[1]['LTeamID'])] = []
            
        # Append opposing team's Elo score to dictionary
        opponent_elo_dicts[str(game[1]['WTeamID'])].append(game[1]['LElo'])
        opponent_elo_dicts[str(game[1]['LTeamID'])].append(game[1]['WElo'])       
    
    # Generate the mean of each dictionary of opponent Elos
    opponent_elo_scores = dict()
    
    for team in opponent_elo_dicts:
        opponent_elo_scores[team] =  np.mean(opponent_elo_dicts[team])
        opponent_elos = pd.DataFrame.from_dict(opponent_elo_scores, orient='index', columns = ['OElo'])

    all_opponent_elos[str(season)] = opponent_elos
    # Loop through the dictionary and create a new DataFrame for each key-value pair
    
    for team, opponent_elo in all_opponent_elos[str(season)].items():
        temp_df = pd.DataFrame(opponent_elo, columns=["Season", team])
        temp_df.loc[:, "Season"] = str(season) 
    
    # Append the new DataFrame to the final DataFrame
    all_opponent_elos_df = pd.concat([all_opponent_elos_df, temp_df], axis=0)
        

In [ ]:
all_opponent_elos_df.reset_index(inplace=True)
all_opponent_elos_df.columns=['TeamID', 'Season', 'OElo']

In [ ]:
all_opponent_elos_df.head()

## Merge the datasets together

In [ ]:
pyth_trimmed = pd.DataFrame(total_points.set_index(['Season', 'TeamID'])['Pythagorean'])
elo_trimmed = elo_scores_unstacked.set_index(['Season', 'TeamID'])
df_features_season_trimmed = df_features_season.set_index(['Season', 'TeamID'])[['WinPct', 'AvgScoringMargin']]
opponent_elo_trimmed = all_opponent_elos_df.set_index(['Season', 'TeamID'])
pyth_by_period_trimmed = pyth_by_period['Trend']
all_statistics = pd.concat([pyth_trimmed, elo_trimmed, df_features_season_trimmed, season_sabermetrics, \
                            opponent_elo_trimmed, pyth_by_period_trimmed], axis=1)

In [ ]:
all_statistics[all_statistics['Trend'].isna()]

In [ ]:
winning_tourney_results = df_tourney_results[['Season', 'WTeamID', 'LTeamID']].merge(all_statistics, 
                                                                                     left_on=['Season', 'WTeamID'], 
                                                                                     right_index=True, how='left')
losing_tourney_results = df_tourney_results[['Season', 'WTeamID', 'LTeamID']].merge(all_statistics, 
                                                                                    left_on=['Season', 'LTeamID'], 
                                                                                    right_index=True, how='left')

In [ ]:
combined_tourney_results = pd.merge(left=winning_tourney_results, 
                                    right=losing_tourney_results, 
                                    on=['Season', 'WTeamID', 'LTeamID'], 
                                    suffixes=['_W', '_L'])

df_seeds[['Season', 'TeamID']] = df_seeds[['Season', 'TeamID']].astype(str)

combined_tourney_results = pd.merge(
    combined_tourney_results,
    df_seeds,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'Seed_W'})

combined_tourney_results = pd.merge(
    combined_tourney_results, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'Seed_L'})

combined_tourney_results.columns

In [ ]:
combined_tourney_results.set_index(['Season', 'WTeamID', 'LTeamID'], inplace=True)

In [ ]:
# Differences 
df_tourney_features = pd.DataFrame(index=combined_tourney_results.index)
df_tourney_features['PtsPerPossDiff'] = combined_tourney_results['PtsPerPoss_W'] - combined_tourney_results['PtsPerPoss_L']
df_tourney_features['OPtsPerPossDiff'] = combined_tourney_results['OPtsPerPoss_W'] - combined_tourney_results['OPtsPerPoss_L']
df_tourney_features['EffectiveFGPctDiff'] = combined_tourney_results['EffectiveFGPct_W'] - combined_tourney_results['EffectiveFGPct_L']
df_tourney_features['AvgScoringMarginDiff'] = combined_tourney_results['AvgScoringMargin_W'] - combined_tourney_results['AvgScoringMargin_L']
df_tourney_features['AssistRateDiff'] = combined_tourney_results['AssistRate_W'] - combined_tourney_results['AssistRate_L']
df_tourney_features['ReboundPctDiff'] = combined_tourney_results['ReboundPct_W'] - combined_tourney_results['ReboundPct_L']
df_tourney_features['StealBlockFoulDiff'] = combined_tourney_results['StealBlockFoul_W'] - combined_tourney_results['StealBlockFoul_L']
df_tourney_features['TORateDiff'] = combined_tourney_results['TORate_W'] - combined_tourney_results['TORate_L']
df_tourney_features['BArcPctDiff'] = combined_tourney_results['BArcPct_W'] - combined_tourney_results['BArcPct_L']
df_tourney_features['FTRateDiff'] = combined_tourney_results['FTRate_W'] - combined_tourney_results['FTRate_L']
df_tourney_features['FTPctDiff'] = combined_tourney_results['FTPct_W'] - combined_tourney_results['FTPct_L']
df_tourney_features['WinPctDiff'] = combined_tourney_results['WinPct_W'] - combined_tourney_results['WinPct_L']
df_tourney_features['EloDiff'] = combined_tourney_results['Elo_W'] - combined_tourney_results['Elo_L']
df_tourney_features['PythagoreanDiff'] = combined_tourney_results['Pythagorean_W'] - combined_tourney_results['Pythagorean_L']
df_tourney_features['OEloDiff'] = combined_tourney_results['OElo_W'] - combined_tourney_results['OElo_L']
df_tourney_features['SeedDiff'] = combined_tourney_results['Seed_W'].astype(int) - combined_tourney_results['Seed_L'].astype(int)
df_tourney_features['TrendDiff'] = combined_tourney_results['Trend_W'] - combined_tourney_results['Trend_L']
df_tourney_features['Gender'] = df_tourney_results['Gender'].values # Used for cross validation


In [ ]:
df_tourney_features.info()

In [ ]:
df_tourney_features['ScoreDiff'] = (df_tourney_results['WScore'] - df_tourney_results['LScore']).values
df_tourney_features['WinA'] = (df_tourney_features['ScoreDiff'] > 0).astype(int)

In [ ]:
df_tourney_features.index.rename(['Season', 'TeamA', 'TeamB'], inplace=True)
df_tourney_features.info()

## Create symmetrical datasets for double the data

In [ ]:
df_tourney_features_symmetrical = df_tourney_features.copy()
df_tourney_features_symmetrical.index.rename(['Season', 'TeamB', 'TeamA'], inplace=True)
df_tourney_features_symmetrical = df_tourney_features_symmetrical * -1
df_tourney_features_symmetrical['Gender'] = df_tourney_features_symmetrical['Gender'] * -1 # Lazy way to reverse any prior changes
df_tourney_features_symmetrical['WinA'] = 0
df_tourney_features_combined = pd.concat([df_tourney_features, df_tourney_features_symmetrical], axis=0)

df_tourney_features_combined.head()

In [ ]:
n_rows = 5; n_cols = 4
figs, axs = plt.subplots(nrows=n_rows, ncols=n_cols, squeeze=True, figsize=(20, 16))

row = 0; col = 0
for feature in df_tourney_features.columns:
    sns.histplot(df_tourney_features[feature], ax=axs[row, col])
    axs[row, col].set_xlabel(feature)
    axs[row, col].axvline(0, color='red')
    row += 1
    if row == n_rows:
        row = 0
        col += 1

plt.tight_layout()
plt.show()

## Scaling variables

In [ ]:
def standard_scale(features, df_train, df_val, df_test=None):    
    mm = MinMaxScaler() 
    
    df_train[features] = pd.DataFrame(mm.fit_transform(df_train[features]), columns = features)
    df_val[features] = pd.DataFrame(mm.transform(df_val[features]), columns = features)
    
    if df_test is not None:
        df_test[features] = pd.DataFrame(mm.transform(df_test[features]), columns = features)
    
    return df_train, df_val, df_test

## Correlation check for variables

In [ ]:
f, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(df_tourney_features_combined.corr(), annot=True)
plt.show()

## Mutual Information Scores

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_classif(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

X_mi = df_tourney_features_combined.drop(['WinA', 'ScoreDiff'], axis=1)
y_mi = df_tourney_features_combined['WinA']
mi_scores = make_mi_scores(X_mi, y_mi)
f, ax = plt.subplots(figsize=(10, 10))
plot_mi_scores(mi_scores)

### Eliminate features with low mutual information scores

In [ ]:
features = ['SeedDiff', 'OEloDiff', 'EloDiff', 'AvgScoringMarginDiff', 'PythagoreanDiff', 'PtsPerPossDiff', 'WinPctDiff', \
            'StealBlockFoulDiff', 'EffectiveFGPctDiff', 'OPtsPerPossDiff', 'TrendDiff', 'FTRateDiff', 'TORateDiff', 'BArcPctDiff']

target = ['WinA']


### Create feature and target variables

In [ ]:
X = df_tourney_features_combined[features]
y = df_tourney_features_combined[target].values

In [ ]:
df_tourney_features_combined.head()

## Create X_test

In [ ]:
sample_submission = data['SampleSubmission2023'].copy()

In [ ]:
sample_submission['Season'] = sample_submission['ID'].apply(lambda x: x.split('_')[0])
sample_submission['TeamA'] = sample_submission['ID'].apply(lambda x: x.split('_')[1])
sample_submission['TeamB'] = sample_submission['ID'].apply(lambda x: x.split('_')[2])

In [ ]:
df_test_statistics = all_statistics.reset_index().copy()
df_test_statistics = df_test_statistics[df_test_statistics['Season'] == '2023'].reset_index(drop=True)


In [ ]:
test_teamA_stats = sample_submission.merge(df_test_statistics, left_on=['Season', 'TeamA'], right_on=['Season', 'level_1'])
test_teamB_stats = sample_submission.merge(df_test_statistics, left_on=['Season', 'TeamB'], right_on=['Season', 'level_1'])

In [ ]:
test_teamA_stats = test_teamA_stats.merge(df_seeds, left_on=['Season', 'TeamA'], right_on=['Season', 'TeamID'], how='left')
test_teamB_stats = test_teamB_stats.merge(df_seeds, left_on=['Season', 'TeamB'], right_on=['Season', 'TeamID'], how='left')

In [ ]:
merged_test_team_stats = test_teamA_stats.merge(test_teamB_stats, on=['Season', 'TeamA', 'TeamB'], suffixes=['_A', '_B'])
merged_test_team_stats['Seed_A'].fillna(17, inplace=True)
merged_test_team_stats['Seed_B'].fillna(17, inplace=True)

In [ ]:
merged_test_team_stats.info()

In [ ]:
X_test = pd.DataFrame()
final_features = ['Seed', 'Pythagorean', 'Elo', 'OElo', 'WinPct', 'AvgScoringMargin', 'PtsPerPoss',
                  'OPtsPerPoss', 'EffectiveFGPct', 'TORate', 'BArcPct', 'FTRate', 'StealBlockFoul', 'Trend']

for feature in final_features:
    X_test[feature + 'Diff'] = merged_test_team_stats[feature + '_A'] - merged_test_team_stats[feature + '_B']

In [ ]:
X_test.tail()

In [ ]:
X_test['Season'] = merged_test_team_stats['ID_A'].apply(lambda x: x.split('_')[0]).values
X_test['TeamA'] = merged_test_team_stats['ID_A'].apply(lambda x: x.split('_')[1]).values
X_test['TeamB'] = merged_test_team_stats['ID_A'].apply(lambda x: x.split('_')[2]).values

In [ ]:
X_test.fillna(0, inplace=True)

In [ ]:
X_test.set_index(['Season', 'TeamA', 'TeamB'], inplace=True)

## Check yourself before you wreck yourself: Plot X_test variables

In [ ]:
n_rows = 4; n_cols = 4
figs, axs = plt.subplots(nrows=n_rows, ncols=n_cols, squeeze=True, figsize=(16, 16))

row = 0; col = 0
for feature in X_test.columns:
    sns.histplot(X_test[feature], ax=axs[row, col])
    axs[row, col].set_xlabel(feature)
    axs[row, col].axvline(0, color='red')
    row += 1
    if row == n_rows:
        row = 0
        col += 1

plt.tight_layout()
plt.show()

## Throwing the Kitchen Sink: Modeling Time

In [ ]:
X.loc[:, 'Gender'] = df_tourney_features_combined.loc[:, 'Gender']
X.loc[:, 'WinA'] = df_tourney_features_combined.loc[:, 'WinA']

In [ ]:
X_trimmed = X.reset_index()[(X.reset_index()['Season'].astype(int) >= 2003) & (X.reset_index()['Season'].astype(int) <= 2016)]

X_trimmed.set_index(['Season', 'TeamA', 'TeamB'], inplace=True)
y_trimmed = X_trimmed['WinA'].values

In [ ]:
groups = X_trimmed.reset_index()['Gender'].astype(str) + '_' + \
         X_trimmed.reset_index()['Season'].astype(str)
gss = GroupShuffleSplit(n_splits=5, test_size=.2)
X_trimmed.drop(['WinA', 'Gender'], axis=1, inplace=True)
X_scaled = pd.DataFrame(MinMaxScaler().fit_transform(X_trimmed[features]))
X_scaled.set_index(groups, inplace=True)

In [ ]:
# Select only seasons 2003-2016 to prevent data leakage (we're validating on seasons 2017-2022).

In [ ]:
'''
def objective(trial):
    xgb_params = dict(
        max_depth=trial.suggest_int("max_depth", 2, 32),
        learning_rate=trial.suggest_float("learning_rate", 0, 1),
        n_estimators=30,
        min_child_weight=trial.suggest_int("min_child_weight", 1, 20),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.01, 1),
        colsample_bynode=trial.suggest_float("colsample_bynode", 0.01, 1),
        colsample_bylevel=trial.suggest_float("colsample_bylevel", 0.01, 1),
        max_delta_step=trial.suggest_int("max_delta_step", 1, 20),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e0, 1e3, log=True),
        gamma=trial.suggest_float("gamma", 1e-6, 1, log=True)
        
    )
    return score_dataset(X_scaled, y, xgb_params)

def score_dataset(X_scaled, y, xgb_params):
    
    classifier = xgb.XGBClassifier(**xgb_params)
    
    score = cross_val_score(classifier, X_scaled, y_trimmed, cv=gss, groups=groups, scoring='neg_brier_score')
    score = -1 * score.mean() + score.std()
    return score

study = optuna.create_study()
study.optimize(objective, n_trials=30)
xgb_params = study.best_params
print(xgb_params)

plot_slice(study)
'''


In [ ]:
# fig = optuna.visualization.plot_param_importances(study)
# fig.show()

In [ ]:
def objective(trial):
    lgbm_params = {
        'reg_alpha': trial.suggest_float("reg_alpha", 1e-6, 1e2, log=True),
        'reg_lambda': trial.suggest_float("reg_lambda", 1e0, 1e3, log=True),
        'num_leaves': 100,
        'max_depth': trial.suggest_int('max_depth', 2, 32),
        'learning_rate': trial.suggest_float("learning_rate", 0, 1),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0, 1),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0, 1),
        'n_estimators': 30,
    }
    return score_dataset(X_scaled, y, lgbm_params)

def score_dataset(X_scaled, y, lgbm_params):
    
    classifier = lgb.LGBMClassifier(**lgbm_params, class_weight='balanced')
    
    score = cross_val_score(classifier, X_scaled, y_trimmed, cv=gss, groups=groups, scoring='neg_brier_score')
    score = -1 * score.mean() + score.std()
    return score

study = optuna.create_study()
study.optimize(objective, n_trials=100)
lgbm_params = study.best_params
print(lgbm_params)

plot_slice(study)

In [ ]:
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
def kfold_reg(df, df_test_=None, plot=False, verbose=0, mode="reg"):
    seasons = df['Season'].unique()
    
    cvs = []
    pred_tests = []
    target = "ScoreDiff" if mode == "reg" else "WinA"
    
    features = ['SeedDiff', 'OEloDiff', 'EloDiff', 'AvgScoringMarginDiff', 'PythagoreanDiff', 'PtsPerPossDiff', 'OPtsPerPossDiff', 'WinPctDiff', \
               'EffectiveFGPctDiff', 'StealBlockFoulDiff', 'FTRateDiff', 'TORateDiff', 'BArcPctDiff', 'TrendDiff']
    
    for season in seasons[14:]:
        if verbose:
            print(f'\nValidating on season {season}')
        else:
            print(f'\nTesting on season {season}')
        
        current_season = int(season)
        
        df_train = df[df['Season'] < season].reset_index(drop=True).copy()
        df_val = df[df['Season'] == season].reset_index(drop=True).copy()
        df_test = df_test_.reset_index(drop=True).copy()
        
        df_train, df_val, df_test = standard_scale(features, df_train, df_val, df_test)
        
        if mode == "reg":
            model = ElasticNet(alpha=1, l1_ratio=0.5)
        else:
            logistic = LogisticRegression(C=100, max_iter=10000, class_weight='balanced')
            lsvc = LinearSVC(max_iter=10000, class_weight='balanced', fit_intercept=False) 
            # I snuck this in because it did really well in validation testing.
            # xgbc = xgb.XGBClassifier(**xgb_params)
            lgbm = lgb.LGBMClassifier(**lgbm_params, class_weight='balanced')
            stacked = StackingClassifier(estimators=[
                                                     ('LinearSVC', lsvc),
                                                     #('XGBoost', xgbc),
                                                     ('LightGBM', lgbm),
                                                     ], 
                                                     final_estimator=logistic, 
                                                     n_jobs=-1)
            model = stacked
            
        model.fit(df_train[features], df_train[target])
        calibrated = CalibratedClassifierCV(model, method='sigmoid', cv=5) 
        calibrated.fit(df_train[features], df_train[target])
        
        if mode == "reg":
            pred = model.predict(df_val[features])
            pred = (pred - pred.min()) / (pred.max() - pred.min())
        else:
            pred = calibrated.predict_proba(df_val[features])[:, 1]
            # pred = (pred - pred.min()) / (pred.max() - pred.min())
            
        if df_test is not None:
            if mode == "reg":
                pred = model.predict(df_test[features])
            else:
                pred_test = calibrated.predict_proba(df_test[features])[:, 1]
                # pred_test = (pred_test - pred_test.min()) / (pred_test.max() - pred_test.min()) 
                    
            pred_tests.append(pred_test)
            
        if plot:
            plt.figure(figsize=(15, 6))
            plt.subplot(1, 2, 1)
            plt.scatter(pred, df_val['ScoreDiff'].values, s=5)
            plt.grid(True)
            plt.subplot(1, 2, 2)
            sns.histplot(pred)
            plt.show()

        if verbose:
            loss = ((df_val['WinA'].values - pred) ** 2).mean()
            cvs.append(loss)
            print(f'\t -> Scored {loss:.4f}')
        
    print(f'\n Local CV is {np.mean(cvs):.4f}')
    
    return pred_tests

In [ ]:
X = df_tourney_features_combined.reset_index()

pred_tests = kfold_reg(X, X_test, plot=True, verbose=1, mode="cls")

In [ ]:
X_test.info()

## Predict on Test Set

In [ ]:
pred_test = np.mean(pred_tests, 0)

## Prepare for automatic submission

In [ ]:
final_submission = pd.DataFrame()
final_submission.index = merged_test_team_stats['ID_A']
final_submission.index.rename('ID', inplace=True)
final_submission['Pred'] = pred_test
final_submission.to_csv('submission.csv')

## Check yourself before you wreck yourself part II: Probability Histogram

In [ ]:
_ = sns.displot(final_submission)
print(final_submission.mean())